In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

2024-11-29 20:23:48.152695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 20:23:48.339552: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 20:23:48.385754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 20:23:48.685181: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 20:23:50.650493: W tensorflow/compiler/tf2

# Importing data

In [2]:
df = pd.read_csv('X_train_Hi5.csv', nrows=10000)

/tmp/ipykernel_59805/1203643383.py:1: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('X_train_Hi5.csv', nrows=10000)


In [3]:
df

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
0,0,01,Sun Jul 14 13:00:02 CEST 2024,20.00,Ain,01073,PIEZOMETRE - MARAIS DE LAVOURS (CEYZERIEU - BR...,['712AH37'],232.00,07004X0046/D6-20,...,5.698947,45.725106,1454.0,1.793055e+09,11.8,992.0,25250,2.9,16.2,High
1,1,01,Sun Jul 14 13:00:02 CEST 2024,35.60,Ain,01363,PIEZOMETRE - GRAVIERE (ST-JEAN-LE-VIEUX - BRGM...,['712GB05'],247.25,06754X0077/F1,...,5.464933,46.210734,1051.0,1.085125e+09,0.6,1786.0,24660,44.5,11,Very High
2,2,01,Sun Jul 14 13:00:02 CEST 2024,35.22,Ain,01244,PIEZOMETRE - BORD AUTOROUTE (MEXIMIEUX - BRGM ...,['040AJ43'],218.77,06993X0226/MEXI_2,...,5.085060,45.812828,69266.0,3.810492e+08,0,8085.0,24890,8.4,7.8,High
3,3,01,Sun Jul 14 13:00:02 CEST 2024,34.20,Ain,01288,PIEZOMETRE - GRENY (PERON - BRGM 01) - BSH,"['516AA00', '516AF00']",499.85,06533X0070/F2,...,5.802841,46.366049,39286.0,3.800911e+08,1.5,2838.0,39700,2.4,5.2,Very High
4,4,01,Sun Jul 14 13:00:02 CEST 2024,37.30,Ain,01422,FORAGE - ENCLOS (TOSSIAT - BRGM 01) - BSH,['507AB00'],260.00,06518X0026/P2,...,5.377265,46.080989,1273.0,1.966631e+07,0.2,1352.0,26180,21.5,9.8,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,10517,6,Fri Jun 28 07:31:38 CEST 2024,NaN,Alpes-Maritimes,6027,collège Romée,NaN,10.00,09994X0536/PZ_CRO,...,7.193913,43.854155,6021.0,1.611474e+08,0.1,52580.0,23940,5.3,7.8,Low
9996,10518,6,Fri Jun 28 07:31:38 CEST 2024,NaN,Alpes-Maritimes,6161,Grand Pré,NaN,14.00,09994X0533/PZ_GDP,...,7.193913,43.854155,6021.0,1.621418e+08,0.1,16779.0,26260,5.2,6,Low
9997,10519,6,Fri Jun 28 07:31:38 CEST 2024,NaN,Alpes-Maritimes,6027,Hippodrome,NaN,1.00,09998X0197/PZ_HIP,...,7.193913,43.854155,6021.0,1.566103e+08,0.1,52580.0,23940,5.3,7.8,High
9998,10520,6,Fri Jun 28 07:31:38 CEST 2024,17.50,Alpes-Maritimes,6033,P33BIS,NaN,76.17,09735X0161/P33BIS,...,7.188727,43.901261,6078.0,1.036715e+09,0,13277.0,23480,35.8,9.5,Very High
